# Notebook Details

In this notebook we will give Long term memory.

We have 2 tools:
- 1. Calculator Tools.
- 2. wikipwdia web search.

We will store old conversion to a list and pass this histoy as context of the query.

As of now the agent can answer only one query. But We will add looping concept here, So that we can interact with the agent as long as we want.

In Previous Notebook we have build it feom scratch. This time we will be using popular freamwork called `langchain` to build the agent.

## Importing Dependencies

In [1]:
# install necessary libraries
# importing necessary libraries
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
import wikipedia


## Defining Tools

In [2]:
# --- Configuration --- #
MODEL_NAME = "llama3.2:latest"
# MEMORY_PATH = "memory_V2.json"
# MAX_MEMORY = 5

In [3]:
# 1. Simple calculator tool
def calculator_tool(query):
    """A simple calculator tool."""
    try:
        # Keep only numbers, operators, parentheses, and dots
        cleaned_expr = query.split('=')[0]  # Remove anything after '='
        cleaned_expr = "".join(cleaned_expr).strip()
        return str(eval(cleaned_expr))
    except Exception as e:
        return f"Error in calculation: {e}"

In [4]:
# # 2. Wikipedia search tool
# def clean_text(text):
#     """Remove HTML tags and excessive whitespace."""
#     # If HTML is present, parse it
#     soup = BeautifulSoup(text, "html.parser")
#     clean = soup.get_text(separator=" ", strip=True)
#     # Remove multiple spaces/newlines
#     clean = re.sub(r"\s+", " ", clean)
#     return clean


# def web_search(query):
#     url = "https://en.wikipedia.org/w/api.php"
#     params = {
#         "action": "query",
#         "list": "search",
#         "srsearch": query,
#         "format": "json"
#     }
#     try:
#         response = requests.get(url, params=params)
#         data = response.json()
#         if "query" in data and "search" in data["query"]:
#             results = []
#             for item in data['query']['search']:
#                 # Remove HTML Tags
#                 clean_text = re.sub(r'<[^>]+>', '', item['snippet'])
#                 results.append(clean_text.strip())
#             return "\n".join(results) if results else "No results found on Wikipedia."
#         else:
#             return "No results found."
#     except Exception as e:
#         return f"Error searching Wikipedia: {e}"

In [5]:
# 2. Wikipedia search tool
def wikipedia_tool(query):
    """Search Wikipedia for a given query."""
    try:
        results = wikipedia.search(query, results=5)
        if not results:
            return "No results found on Wikipedia."
        
        summaries = []
        for title in results:
            summary = wikipedia.summary(title, sentences=1)
            summaries.append(f"{title}: {summary}")
        
        return "\n".join(summaries)
    except Exception as e:
        return f"Error searching Wikipedia: {e}"

In [6]:
# wraping the tools in langchain tool objects
tools = [
    Tool(
        name="Calculator",
        func=calculator_tool,
        description="A simple calculator tool that can perform basic arithmetic operations. Input should be a mathematical expression like '2 + 2'."
    ),
    Tool(
        name="Wikipedia Search",
        func=wikipedia_tool,
        description="Search Wikipedia for a given query. Input should be a search term."
    )
]

## Defining the LLM model

In [7]:
llm = Ollama(model=MODEL_NAME)

C:\Users\Karmakar\AppData\Local\Temp\ipykernel_20576\4231901401.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=MODEL_NAME)


## Defining Memory

In [8]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

C:\Users\Karmakar\AppData\Local\Temp\ipykernel_20576\1192691676.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


## Defining Agent

In [11]:
# Defining system prompt for the agent
system_prompt = f"""
    You are a helpful agent with tools:
    1. CALCULATE: for math problems
    2. SEARCH: for Wikipedia searches

    Decide which tool to use based on the user input.
    Respond only with one of the following formats:
    CALCULATE: <expression>
    SEARCH: <query>
    ANSWER: <your direct answer>

    User: <query>
    Agent:"""
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="conversational-react-description", # conversational agent with reasoning
    verbose=True,
    memory=memory,
    agent_kwargs={"system_prompt": system_prompt}
)

In [12]:
# Creating the conversational loop
print("🤖 Agent started. Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    print(f"You: {user_input}")
    if user_input.lower() in ["exit", "quit", "q"]:
        print("Exiting the agent. Goodbye!")
        break
    
    response = agent.run(user_input)
    print('# ------- Response ------- #')
    print(f"Agent: {response}")
    print('# ------------------------ #')

🤖 Agent started. Type 'exit' to quit.
You: What is the sum of first 5 odd numbers


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 1 + 3 + 5 + 7 + 9
Observation: 25
Thought:Thought: Do I need to use a tool? No
AI: The sum of the first 5 odd numbers is 25.

> Finished chain.
# ------- Response ------- #
Agent: The sum of the first 5 odd numbers is 25.
# ------------------------ #
You: How old is the current prime minister of India?


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: The current Prime Minister of India is Narendra Modi. He has been serving as the Prime Minister since May 2014 and is the leader of the Bharatiya Janata Party (BJP).

> Finished chain.
# ------- Response ------- #
Agent: The current Prime Minister of India is Narendra Modi. He has been serving as the Prime Minister since May 2014 and is the leader of the Bharatiya Janata Party (BJP).
# ------------------------ #
You: W